In [9]:
import kagglehub

In [10]:
!pip install --upgrade typing_extensions

In [11]:
path = kagglehub.dataset_download("ritesaluja/bank-note-authentication-uci-data")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Rand Rothstein\.cache\kagglehub\datasets\ritesaluja\bank-note-authentication-uci-data\versions\1


In [14]:
import pandas as pd 
data = pd.read_csv(os.path.join(path,'BankNote_Authentication.csv'))


In [8]:
import os

path = r'C:\Users\Rand Rothstein\.cache\kagglehub\datasets\ritesaluja\bank-note-authentication-uci-data\versions\1'
print(os.listdir(path))


['BankNote_Authentication.csv']


In [15]:
data.head()

,variance,skewness,curtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [16]:
x=data.iloc[:,:-1]
y=data.iloc[:,-1]

In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)

In [20]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(x_train,y_train)


RandomForestClassifier()

In [23]:
from sklearn.metrics import accuracy_score,classification_report

y_pred=classifier.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       232
           1       0.98      0.99      0.99       180

    accuracy                           0.99       412
   macro avg       0.99      0.99      0.99       412
weighted avg       0.99      0.99      0.99       412



In [52]:
import pickle
pickle_out = open('model.pkl','wb')
pickle.dump(classifier,pickle_out)
pickle_out.close()

In [33]:
print(x_test)
dftest=pd.DataFrame(x_test)
dftest.to_csv('test.csv',index=False)

      variance   skewness  curtosis  entropy
1023  -1.77130 -10.766500  10.21840 -1.00430
642    5.13210  -0.031048   0.32616  1.11510
1196  -2.01490   3.687400  -1.93850 -3.89180
31     1.48840   3.627400   3.30800  0.48921
253    5.28680   3.257000  -1.37210  1.16680
...        ...        ...       ...      ...
986    0.84546   3.482600  -3.63070 -1.39610
271    5.39150   9.994600  -3.80810 -3.36420
1266  -1.43770  -1.432000   2.11440  0.42067
769   -0.89409   3.199100  -1.82190 -2.94520
634    6.82480   5.218700  -2.54250  0.54610

[412 rows x 4 columns]


***Flask API***

In [ ]:
from flask import Flask,request
import numpy as np
from flasgger import Swagger

app=Flask(__name__)
Swagger(app)
pickle_in=open('model.pkl','rb')
classifier_loaded=pickle.load(pickle_in)

@app.route('/')                           #'/' refers to the root URL of the web application (like http://localhost:5000/).
def welcome():
    return "Welcome Rand"

@app.route('/predict')
def predict_note_authentication():
    """
    Let's Authenticate the Bank Note 
    ---
    parameters:
      - name: Variance
        in: query
        type: number
        required: true
      - name: Skewness
        in: query
        type: number
        required: true
      - name: Curtosis
        in: query
        type: number
        required: true
      - name: entropy
        in: query
        type: number
        required: true
    responses:
      200:
        description: The output values
    """

    variance=request.args.get('Variance')
    skewness=request.args.get('Skewness')
    curtosis=request.args.get('Curtosis')
    entropy=request.args.get('entropy')
    
    prediction = classifier_loaded.predict([[variance, skewness, curtosis, entropy]])

    return "Predicted Class is "+ str(prediction)

@app.route('/predict_file',methods=['POST'])
def predict_note_authentication_file():

    df_test=pd.read_csv(request.files.get("file"))
    prediction=classifier_loaded.predict(df_test)

    return "Predicted Class is for file"+ str(list(prediction))
if __name__=='__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off



 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/May/2025 17:55:22] "GET / HTTP/1.1" 200 -
C:\Users\Rand Rothstein\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
127.0.0.1 - - [12/May/2025 17:55:31] "GET /predict?Variance=2.3&Skewness=4.5&Curtosis=0.1&entropy=-1.2 HTTP/1.1" 200 -


In [53]:
with open('model.pkl', 'rb') as f:
    classifier_loaded = pickle.load(f)
print(type(classifier_loaded))


<class 'sklearn.ensemble._forest.RandomForestClassifier'>


In [ ]:
import requests

url = 'http://127.0.0.1:5000/predict_file'
files = {'File': open('test.csv', 'rb')}

response = requests.post(url, files=files)
print(response.text)
